# Reproducibilidad de experimentos

In [1]:
import sys
import os

import torch
import random

import torchaudio.transforms    as T
import torch.optim              as optim
import pandas                   as pd
import numpy                    as np

from torchvision       import transforms
from torch.utils.data  import DataLoader
from torch             import nn
from sklearn.metrics   import balanced_accuracy_score, recall_score, precision_score, mean_absolute_error, average_precision_score

sys.path.append(os.path.abspath(os.path.join('..','05-Train-Test')))
from utilit_train_test import make_weights_for_balanced_classes

sys.path.append(os.path.abspath(os.path.join('..','..','iESPnet_SRC_main','utilities')))
from Generator         import SeizureDatasetLabelTimev2, permute_spec, smoothing_label
from Model             import iESPnet
from TrainEval         import train_model_v2, test_model_v2, get_performance_indices
from IO                import get_spectrogram_2

sys.path.append(os.path.abspath(os.path.join('../../..','03 Dynamic-Spatial-Filtering')))
from models            import DynamicSpatialFilter

torch.set_printoptions(precision=10)

# set the seed for reproducibility
torch.manual_seed(0)
random.seed(0)

In [2]:
# direccion donde se encuentran los espectrogramas 
SPE_DIR        = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/allfiles_metadata.csv'

df_meta        = pd.read_csv(meta_data_file)

In [3]:
# Variables iESPnet
FREQ_MASK_PARAM    = 10
TIME_MASK_PARAN    = 20
N_CLASSES          = 1
learning_rate      = 1e-3
batch_size         = 64    #128
epochs             = 20
num_workers        = 4

save_path          = 'models_DSF_iESPnet_prueba/'
patients           = df_meta['rns_id'].unique().tolist()

In [4]:
# Variables DSF
denoising          = 'autoreject'   # 'autoreject' 'data_augm' 
model              = 'stager_net'
dsf_type           = 'dsfd'         # 'dsfd' 'dsfm_st'
mlp_input          = 'log_diag_cov'
dsf_soft_thresh    = False
dsf_n_out_channels = None
n_channels         = 4

In [5]:
# hiperparametros iESPnet
hparams = {
           "n_cnn_layers" : 3,
           "n_rnn_layers" : 3,
           "rnn_dim"      : [150, 100, 50],
           "n_class"      : N_CLASSES,
           "out_ch"       : [8,8,16],
           "dropout"      : 0.3,
           "learning_rate": learning_rate,
           "batch_size"   : batch_size,
           "num_workers"  : num_workers,
           "epochs"       : epochs
          }

In [6]:
# define train y test de df_meta
test_id  = ['PIT-RNS1090', 'PIT-RNS8973', 'PIT-RNS1438', 'PIT-RNS8326', 'PIT-RNS3016']
vali_id  = ['PIT-RNS1603', 'PIT-RNS1556', 'PIT-RNS1534', 'PIT-RNS6989', 'PIT-RNS2543', 'PIT-RNS7168', 'PIT-RNS6762']

In [7]:
train_df = df_meta.copy() # hace falta resetear el indice de train_df?
test_df  = pd.DataFrame()
vali_df  = pd.DataFrame()

In [8]:
for s in range (len(test_id)):
    test_df = pd.concat([test_df, df_meta[df_meta['rns_id'] == test_id[s]]])
    test_df.reset_index(drop=True, inplace=True)
    train_df.drop(train_df[train_df['rns_id'] == test_id[s]].index, inplace = True)

for s in range(len(vali_id)):
    vali_df=pd.concat([vali_df, df_meta[df_meta['rns_id'] == vali_id[s]]])
    vali_df.reset_index(drop=True, inplace=True)
    train_df.drop(train_df[train_df['rns_id'] == vali_id[s]].index, inplace = True)

train_df.reset_index(drop=True, inplace=True)

In [9]:
# experimentos que se van a realizar
experiments_1 = ['exp1','exp2','exp3']
experiments_2 = ['.1','.2','.3']

In [10]:
s = 0
j = 0

In [11]:
model1 = DynamicSpatialFilter(
                              n_channels, 
                              mlp_input            = mlp_input, 
                              n_out_channels       = dsf_n_out_channels, 
                              apply_soft_thresh    = dsf_soft_thresh
                             )

In [12]:
model2 = iESPnet(
                 hparams['n_cnn_layers'],
                 hparams['n_rnn_layers'],
                 hparams['rnn_dim'],
                 hparams['n_class'],
                 hparams['out_ch'],
                 hparams['dropout'],
                )

In [13]:
save_runs        = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/runs/'
save_models      = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/models/'
save_predictions = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/results/'
save_figs        = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/figs/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

if not os.path.exists(save_runs):
    os.makedirs(save_runs)

if not os.path.exists(save_models):
    os.makedirs(save_models)

if not os.path.exists(save_predictions):
    os.makedirs(save_predictions)

if not os.path.exists(save_figs):
    os.makedirs(save_figs)

In [14]:
print('Running training for: ' + experiments_1[s] +  experiments_2[j])

Running training for: exp1.1


In [15]:
# Dataloaders creados
train_data = SeizureDatasetLabelTimev2(
                                       file             = train_df,
                                       root_dir         = SPE_DIR,
                                       transform        = None, 
                                       target_transform = smoothing_label(),
                                      )

In [16]:
# testing data should be balanced, just be "as it is"
test_data  = SeizureDatasetLabelTimev2(
                                       file             = test_df,
                                       root_dir         = SPE_DIR,
                                       transform        = None,
                                       target_transform = smoothing_label()  
                                      )

In [17]:
# validation data should be balanced, just be "as it is"
vali_data  = SeizureDatasetLabelTimev2(
                                       file             = vali_df,
                                       root_dir         = SPE_DIR,
                                       transform        = None,
                                       target_transform = smoothing_label()  
                                      )

In [18]:
# data augmentation 
transform_train = transforms.Compose([
                                      T.FrequencyMasking(FREQ_MASK_PARAM),
                                      T.TimeMasking(TIME_MASK_PARAN), 
                                      permute_spec()                                                                     
                                    ])

In [19]:
weights = make_weights_for_balanced_classes(train_df, [0,1], n_concat=1)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

In [20]:
outputfile = save_models + 'model_' + str(experiments_1[s] + experiments_2[j])

In [21]:
'''

avg_train_losses, train_accs, avg_valid_losses, valid_accs = train_model_v2(
                                                                            model1, 
                                                                            model2, 
                                                                            hparams, 
                                                                            epochs, 
                                                                            train_data, 
                                                                            vali_data, 
                                                                            transform_train, 
                                                                            sampler, 
                                                                            outputfile,
                                                                            experiments_1[s],
                                                                            experiments_2[j]
                                                                           )

'''

'\n\navg_train_losses, train_accs, avg_valid_losses, valid_accs = train_model_v2(\n                                                                            model1, \n                                                                            model2, \n                                                                            hparams, \n                                                                            epochs, \n                                                                            train_data, \n                                                                            vali_data, \n                                                                            transform_train, \n                                                                            sampler, \n                                                                            outputfile,\n                                                                            experiments_1[s],\n                           

In [22]:
avg_train_losses = []
train_accs       = []

In [23]:
avg_valid_losses = [] 
valid_accs       = []

In [24]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using {} device'.format(device))

Using cuda device


In [25]:
# following pytorch suggestion to speed up training
torch.backends.cudnn.benchmark     = False # cambio para reproducibilidad
torch.backends.cudnn.deterministic = True

In [26]:
kwargs = {'num_workers': hparams["num_workers"], 'pin_memory': True} if use_cuda else {}

train_loader = DataLoader(train_data, batch_size = hparams["batch_size"], sampler = sampler, **kwargs)
valid_loader = DataLoader(vali_data, batch_size = hparams["batch_size"], shuffle = True, **kwargs)

In [27]:
#move model1 to device
model1.to(device)

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [28]:
#move model2 to device
model2.to(device)

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [29]:
print('Num Model Parameters', sum([param1.nelement() for param1 in model1.parameters()]))
print('Num Model Parameters', sum([param2.nelement() for param2 in model2.parameters()]))

Num Model Parameters 120
Num Model Parameters 1654837


In [30]:
optimizer1 = optim.AdamW(model1.parameters(), hparams['learning_rate'], weight_decay=1e-4)
optimizer2 = optim.AdamW(model2.parameters(), hparams['learning_rate'], weight_decay=1e-4)

scheduler1 = optim.lr_scheduler.OneCycleLR(
                                           optimizer1, 
                                           max_lr          = hparams['learning_rate'], 
                                           steps_per_epoch = int(len(train_loader)),
                                           epochs          = hparams['epochs'],
                                           anneal_strategy = 'linear'
                                          )

scheduler2 = optim.lr_scheduler.OneCycleLR(
                                           optimizer2, 
                                           max_lr          = hparams['learning_rate'], 
                                           steps_per_epoch = int(len(train_loader)*2),
                                           epochs          = hparams['epochs'],
                                           anneal_strategy = 'linear'
                                          )
      
criterion = nn.BCEWithLogitsLoss().to(device)

In [31]:
epoch = 1

In [32]:
'''

train_losses, train_aucpr = training_DSF_iESPnet(
                                                 model1, 
                                                 model2, 
                                                 device, 
                                                 train_loader, 
                                                 transform_train, 
                                                 criterion, 
                                                 optimizer1, 
                                                 optimizer2, 
                                                 scheduler1, 
                                                 scheduler2, 
                                                 epoch,
                                                 experiment_1,
                                                 experiment_2
                                                )

'''                                                

'\n\ntrain_losses, train_aucpr = training_DSF_iESPnet(\n                                                 model1, \n                                                 model2, \n                                                 device, \n                                                 train_loader, \n                                                 transform_train, \n                                                 criterion, \n                                                 optimizer1, \n                                                 optimizer2, \n                                                 scheduler1, \n                                                 scheduler2, \n                                                 epoch,\n                                                 experiment_1,\n                                                 experiment_2\n                                                )\n\n'

In [33]:
experiment_1 = experiments_1[s]
experiment_2 = experiments_2[j]

In [34]:
# create spectrogram
ECOG_SAMPLE_RATE = 250
ECOG_CHANNELS    = 4
TT               = 1000 # window length
SPEC_WIN_LEN     = int(ECOG_SAMPLE_RATE * TT / 1000 ) # win size
overlap          = 500 
SPEC_HOP_LEN     = int(ECOG_SAMPLE_RATE * (TT - overlap) / 1000) # Length of hop between windows.
SPEC_NFFT        = 500  # to see changes in 0.5 reso
if   experiment_2 == '.1':  
    top_db       = 40.0
elif experiment_2 == '.2':
    top_db       = 60.0
elif experiment_2 == '.3':
    top_db       = 80.0

In [35]:
train_loss   = 0.0
train_losses = []
cont         = 0

In [36]:
model1.train()

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [37]:
model2.train()

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [38]:
for batch_idx, _data in enumerate(train_loader):
    #if batch_idx == 1:  # Saltar la primera iteración
        #continue

    cont+=1
    eeg, labels = _data 
    eeg, labels = eeg.to(device), labels.to(device)

    # Zero the gradients
    optimizer1.zero_grad(set_to_none=True)
    optimizer2.zero_grad(set_to_none=True)

    # Perform forward pass to DSF
    outputs1 = model1(eeg)  # (batch, n_class)
    outputs1 = outputs1.squeeze(1)
    outputs1 = outputs1.to('cpu')

    # create spectrogram from outputs1
    spectrograms = get_spectrogram_2(outputs1, device, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN, top_db)
    spectrograms = torch.from_numpy(spectrograms)
    
    spectrograms_transformed = transform_train(spectrograms)

    spectrograms2train       = torch.cat((spectrograms, spectrograms_transformed), axis=0)
    spectrograms2train       = spectrograms2train.to(device)

    labels2train = torch.cat((labels, labels), axis=0)

    outputs2 = model2(spectrograms2train)

    m     = nn.Sigmoid()
    probs = m(outputs2)
    
    y_true  = torch.max(labels2train, dim = 1)[0]
    y_pred  = torch.max(probs, dim = 1)[0]
    
    if cont == 1:
        Y_true = y_true
        Y_pred = y_pred
    else:                
        Y_true = torch.cat((Y_true, y_true), axis=0)
        Y_pred = torch.cat((Y_pred, y_pred), axis=0)

    # Compute loss
    loss = criterion(outputs2, labels2train)

    # Perform backward pass
    loss.backward()
    train_loss += loss.item()
    
    # Perform optimization
    optimizer1.step()
    optimizer2.step()
    scheduler1.step()
    scheduler2.step()
    
    # record training loss
    train_losses.append(loss.item())
    del _data
    torch.cuda.empty_cache()
    
    break

OutOfMemoryError: CUDA out of memory. Tried to allocate 502.00 MiB. GPU 

In [ ]:
'''for batch_idx, _data in enumerate(train_loader):
    cont+=1
    eeg, labels = _data 
    eeg, labels = eeg.to(device), labels.to(device)
    break'''

'for batch_idx, _data in enumerate(train_loader):\n    cont+=1\n    eeg, labels = _data \n    eeg, labels = eeg.to(device), labels.to(device)\n    break'

In [ ]:
'''# Zero the gradients
optimizer1.zero_grad(set_to_none=True)
optimizer2.zero_grad(set_to_none=True)'''

'# Zero the gradients\noptimizer1.zero_grad(set_to_none=True)\noptimizer2.zero_grad(set_to_none=True)'

utilizando la semilla random.seed el data loader siempre decide por el mismo batchsize

In [ ]:
'''eeg[0,0,:]'''

'eeg[0,0,:]'

In [ ]:
'''# Perform forward pass to DSF
outputs1 = model1(eeg)  # (batch, n_class)
outputs1 = outputs1.squeeze(1)'''

'# Perform forward pass to DSF\noutputs1 = model1(eeg)  # (batch, n_class)\noutputs1 = outputs1.squeeze(1)'

In [ ]:
'''outputs1[0,0,:]'''

'outputs1[0,0,:]'

In [ ]:
'''if   experiment_1 == 'exp1': # sin normalizacion 
    pass
elif experiment_1 == 'exp2': # normalizacion por canal
    mean     = outputs1.mean(dim=2, keepdim=True)
    std      = outputs1.std(dim=2, keepdim=True)

    outputs1 = (outputs1 - mean) / std

elif experiment_1 == 'exp3': # normalizacion global
    outputs1 = (outputs1 - outputs1.mean()) / outputs1.std()'''

"if   experiment_1 == 'exp1': # sin normalizacion \n    pass\nelif experiment_1 == 'exp2': # normalizacion por canal\n    mean     = outputs1.mean(dim=2, keepdim=True)\n    std      = outputs1.std(dim=2, keepdim=True)\n\n    outputs1 = (outputs1 - mean) / std\n\nelif experiment_1 == 'exp3': # normalizacion global\n    outputs1 = (outputs1 - outputs1.mean()) / outputs1.std()"

In [ ]:
'''outputs1 = outputs1.to('cpu')'''

"outputs1 = outputs1.to('cpu')"

In [ ]:
'''outputs1[0,0,:]'''

'outputs1[0,0,:]'

In [ ]:
'''# create spectrogram from outputs1
spectrograms = get_spectrogram_2(outputs1, device, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN, top_db)'''

'# create spectrogram from outputs1\nspectrograms = get_spectrogram_2(outputs1, device, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN, top_db)'

In [ ]:
'''spectrograms[0,0,:,:]'''

'spectrograms[0,0,:,:]'

In [ ]:
'''spectrograms = torch.from_numpy(spectrograms)'''

'spectrograms = torch.from_numpy(spectrograms)'

In [ ]:
'''spectrograms[0,0,:,:]'''

'spectrograms[0,0,:,:]'

In [ ]:
'''spectrograms_transformed = transform_train(spectrograms)'''

'spectrograms_transformed = transform_train(spectrograms)'

In [ ]:
'''spectrograms_transformed[0,0,:,:]'''

'spectrograms_transformed[0,0,:,:]'

In [ ]:
'''spectrograms2train = torch.cat((spectrograms, spectrograms_transformed), axis=0)'''

'spectrograms2train = torch.cat((spectrograms, spectrograms_transformed), axis=0)'

In [ ]:
'''spectrograms2train[0,0,:,:]'''

'spectrograms2train[0,0,:,:]'

In [ ]:
'''spectrograms2train = spectrograms2train.to(device)'''

'spectrograms2train = spectrograms2train.to(device)'

In [ ]:
'''spectrograms2train[0,0,:,:]'''

'spectrograms2train[0,0,:,:]'

In [ ]:
'''labels2train = torch.cat((labels, labels), axis=0)'''

'labels2train = torch.cat((labels, labels), axis=0)'

In [ ]:
'''outputs2 = model2(spectrograms2train)'''

'outputs2 = model2(spectrograms2train)'

In [ ]:
'''outputs2[0,:]  '''

'outputs2[0,:]  '

In [ ]:
'''m     = nn.Sigmoid()
probs = m(outputs2)'''

'm     = nn.Sigmoid()\nprobs = m(outputs2)'

In [ ]:
'''probs[0,:]'''

'probs[0,:]'

In [ ]:
'''y_true  = torch.max(labels2train, dim = 1)[0]
y_pred  = torch.max(probs, dim = 1)[0]'''

'y_true  = torch.max(labels2train, dim = 1)[0]\ny_pred  = torch.max(probs, dim = 1)[0]'

In [ ]:
'''if cont == 1:
    Y_true = y_true
    Y_pred = y_pred

else:                
    Y_true = torch.cat((Y_true, y_true), axis=0)
    Y_pred = torch.cat((Y_pred, y_pred), axis=0)'''

'if cont == 1:\n    Y_true = y_true\n    Y_pred = y_pred\n\nelse:                \n    Y_true = torch.cat((Y_true, y_true), axis=0)\n    Y_pred = torch.cat((Y_pred, y_pred), axis=0)'

In [ ]:
'''y_true'''

'y_true'

In [ ]:
'''y_pred '''

'y_pred '

In [ ]:
'''loss = criterion(outputs2, labels2train)'''

'loss = criterion(outputs2, labels2train)'

In [ ]:
'''loss'''

'loss'

In [ ]:
'''loss.backward()
train_loss += loss.item()'''

'loss.backward()\ntrain_loss += loss.item()'

In [ ]:
'''# Perform optimization
optimizer1.step()
optimizer2.step()
scheduler1.step()
scheduler2.step()'''

'# Perform optimization\noptimizer1.step()\noptimizer2.step()\nscheduler1.step()\nscheduler2.step()'

In [ ]:
'''# record training loss
train_losses.append(loss.item())'''

'# record training loss\ntrain_losses.append(loss.item())'

In [ ]:
'''del _data
torch.cuda.empty_cache()'''

'del _data\ntorch.cuda.empty_cache()'

In [ ]:
y_val_true, val_pred = Y_true.to('cpu').detach().numpy(), Y_pred.to('cpu').detach().numpy()

In [ ]:
y_val_true

array([1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 1.])

In [ ]:
val_pred 

array([0.55797106, 0.54620266, 0.5524    , 0.5542989 , 0.5525311 ,
       0.56108165, 0.5778388 , 0.55276626, 0.5605448 , 0.55055296,
       0.545822  , 0.5618415 , 0.55384964, 0.5609321 , 0.5619036 ,
       0.5702581 , 0.57056797, 0.56290585, 0.5540417 , 0.56938565,
       0.57902074, 0.5796253 , 0.55720246, 0.5537558 , 0.55639726,
       0.5757947 , 0.5582665 , 0.5702161 , 0.5476137 , 0.5592189 ,
       0.5790522 , 0.5578448 , 0.5698016 , 0.55972767, 0.5554231 ,
       0.5547875 , 0.5514725 , 0.55154866, 0.55223924, 0.56443137,
       0.56661147, 0.5795406 , 0.5634781 , 0.57123744, 0.5551256 ,
       0.57297796, 0.57063806, 0.56352264, 0.56227785, 0.5587263 ,
       0.5673289 , 0.56358534, 0.56178737, 0.5587469 , 0.5516604 ,
       0.5812343 , 0.55771023, 0.5524937 , 0.54951924, 0.5840419 ,
       0.56415915, 0.5678297 , 0.55195683, 0.5747315 , 0.5710066 ,
       0.5563779 , 0.55663514, 0.5959412 , 0.56611717, 0.55511135,
       0.57873577, 0.5805522 , 0.5724227 , 0.550469  , 0.55015

In [ ]:
np.mean(train_losses)

np.float64(0.693660666420691)

In [ ]:
train_aucpr = average_precision_score(y_val_true,val_pred)

In [ ]:
train_aucpr

np.float64(0.7191555105653817)

In [ ]:
'''

valid_losses, valid_aucpr = validate_v2(
                                        model1,
                                        model2,
                                        device, 
                                        valid_loader, 
                                        criterion, 
                                        epoch,
                                        experiment_1,
                                        experiment_2
                                       )

'''

'\n\nvalid_losses, valid_aucpr = validate_v2(\n                                        model1,\n                                        model2,\n                                        device, \n                                        valid_loader, \n                                        criterion, \n                                        epoch,\n                                        experiment_1,\n                                        experiment_2\n                                       )\n\n'

In [ ]:
ECOG_SAMPLE_RATE = 250
ECOG_CHANNELS    = 4
TT               = 1000 
SPEC_WIN_LEN     = int(ECOG_SAMPLE_RATE * TT / 1000 ) 
overlap          = 500 
SPEC_HOP_LEN     = int(ECOG_SAMPLE_RATE * (TT - overlap) / 1000) 
SPEC_NFFT        = 500
if   experiment_2 == '.1':  
    top_db       = 40.0
elif experiment_2 == '.2':
    top_db       = 60.0
elif experiment_2 == '.3':
    top_db       = 80.0

In [ ]:
valid_losses = []
cont         = 0

In [ ]:
model1.eval()

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [ ]:
model2.eval()

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [ ]:
with torch.no_grad():
    for eeg, labels in valid_loader:
        cont+=1
    
        eeg, labels = eeg.to(device), labels.to(device)
        break

In [ ]:
eeg[0,0,:]

tensor([1.2740582228, 1.8588944674, 2.5337054729,  ..., 1.3190456629,
        1.0941085815, 0.6892219782], device='cuda:0')

In [ ]:
outputs1    = model1(eeg)
outputs1    = outputs1.squeeze(1)

In [ ]:
outputs1[0,0,:]

tensor([-0.1259495020, -0.0817473531, -0.3597738743,  ...,
        -0.1269418001, -0.3000434637, -0.1780759096], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [ ]:
if   experiment_1 == 'exp1': # sin normalizacion 
    pass
elif experiment_1 == 'exp2': # normalizacion por canal
    mean     = outputs1.mean(dim=2, keepdim=True)
    std      = outputs1.std(dim=2, keepdim=True)

    outputs1 = (outputs1 - mean) / std

elif experiment_1 == 'exp3': # normalizacion global
    outputs1 = (outputs1 - outputs1.mean()) / outputs1.std()

In [ ]:
outputs1 = outputs1.to('cpu')

In [ ]:
outputs1[0,0,:]

tensor([-0.1259495020, -0.0817473531, -0.3597738743,  ...,
        -0.1269418001, -0.3000434637, -0.1780759096], grad_fn=<SliceBackward0>)

In [ ]:
spectrograms = get_spectrogram_2(outputs1, device, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN, top_db)

In [ ]:
spectrograms[0,0,:,:]

array([[15.629631, 37.139008, 35.388412, ..., 37.550705, 36.55433 ,
        32.092293],
       [15.629631, 36.658447, 35.26042 , ..., 37.148487, 36.000847,
        29.547777],
       [22.065676, 35.139805, 35.05574 , ..., 35.92053 , 34.417027,
        15.629631],
       ...,
       [15.629631, 15.629631, 15.629631, ..., 15.629631, 15.629631,
        19.849653],
       [15.629631, 15.629631, 15.629631, ..., 15.629631, 15.629631,
        18.304625],
       [15.629631, 15.629631, 15.629631, ..., 15.629631, 15.629631,
        15.629631]], dtype=float32)

In [ ]:
spectrograms = torch.from_numpy(spectrograms)
spectrograms = spectrograms.to(device)
spectrograms[0,0,:,:] #36.8055648804

tensor([[15.6296310425, 37.1390075684, 35.3884124756,  ...,
         37.5507049561, 36.5543289185, 32.0922927856],
        [15.6296310425, 36.6584472656, 35.2604217529,  ...,
         37.1484870911, 36.0008468628, 29.5477771759],
        [22.0656757355, 35.1398048401, 35.0557403564,  ...,
         35.9205284119, 34.4170265198, 15.6296310425],
        ...,
        [15.6296310425, 15.6296310425, 15.6296310425,  ...,
         15.6296310425, 15.6296310425, 19.8496532440],
        [15.6296310425, 15.6296310425, 15.6296310425,  ...,
         15.6296310425, 15.6296310425, 18.3046245575],
        [15.6296310425, 15.6296310425, 15.6296310425,  ...,
         15.6296310425, 15.6296310425, 15.6296310425]], device='cuda:0')

In [ ]:
outputs2 = model2(spectrograms)

In [ ]:
outputs2[0,:] #0.0059469901

tensor([ 0.0059469752, -0.0102297068, -0.0369858481, -0.0413380079,
        -0.0359258093, -0.0368227400, -0.0326419882, -0.0314023085,
        -0.0221088491, -0.0265791081, -0.0238872878, -0.0272487439,
        -0.0254285522, -0.0258744024, -0.0237513222, -0.0270480029,
        -0.0193257295, -0.0209990256, -0.0188277774, -0.0170827247,
        -0.0242496096, -0.0280124061, -0.0205696933, -0.0219697170,
        -0.0228655450, -0.0253660195, -0.0217879824, -0.0237421431,
        -0.0273988135, -0.0299718417, -0.0305851512, -0.0261827819,
        -0.0227456577, -0.0185999684, -0.0277628042, -0.0258902647,
        -0.0268729813, -0.0276423581, -0.0333953910, -0.0243354253,
        -0.0255109482, -0.0347631983, -0.0294069909, -0.0263698511,
        -0.0280011408, -0.0205817483, -0.0239039101, -0.0339091308,
        -0.0322487913, -0.0349185579, -0.0409880020, -0.0427820124,
        -0.0462918989, -0.0479957722, -0.0471253358, -0.0453363918,
        -0.0438458286, -0.0451215245, -0.0424111

In [ ]:
m = nn.Sigmoid()
probs = m(outputs2)

In [ ]:
probs[0,:] #0.5033283234

tensor([0.5014867187, 0.4974426329, 0.4907546043, 0.4896669686, 0.4910195172,
        0.4907953739, 0.4918402433, 0.4921500385, 0.4944730103, 0.4933556020,
        0.4940285087, 0.4931882322, 0.4936432540, 0.4935317338, 0.4940624237,
        0.4932383895, 0.4951687455, 0.4947504103, 0.4952931702, 0.4957293868,
        0.4939378500, 0.4929973483, 0.4948577583, 0.4945077598, 0.4942838550,
        0.4936588109, 0.4945532382, 0.4940646887, 0.4931507111, 0.4925076067,
        0.4923542738, 0.4934546649, 0.4943138063, 0.4953501523, 0.4930596948,
        0.4935277700, 0.4932821393, 0.4930898249, 0.4916518927, 0.4939164519,
        0.4936226308, 0.4913100302, 0.4926487505, 0.4934079349, 0.4930001795,
        0.4948547781, 0.4940243065, 0.4915235341, 0.4919385016, 0.4912712574,
        0.4897544384, 0.4893061519, 0.4884291291, 0.4880033731, 0.4882208109,
        0.4886678457, 0.4890402853, 0.4887215495, 0.4893987477, 0.4897204041,
        0.4900880754, 0.4896622598, 0.4882570803, 0.4875197411, 

In [ ]:
if len(probs.shape) == 1:
    probs.unsqueeze_(0)
    outputs2.unsqueeze_(0)

y_true  = torch.max(labels, dim =1)[0]
y_pred  = torch.max(probs, dim=1)[0]

if cont == 1:
    Y_true = y_true
    Y_pred= y_pred

else:                
    Y_true = torch.cat((Y_true, y_true), axis=0)
    Y_pred = torch.cat((Y_pred, y_pred), axis=0)

In [ ]:
loss = criterion(outputs2, labels)
valid_losses.append(loss.item())

In [ ]:
loss

tensor(0.6773409259, device='cuda:0', dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [ ]:
del eeg, loss
torch.cuda.empty_cache()    

In [ ]:
valid_aucpr = average_precision_score(Y_true.to('cpu').detach().numpy(), Y_pred.to('cpu').detach().numpy())

In [ ]:
np.mean(valid_losses), valid_aucpr

(np.float64(0.6773409258907889), np.float64(0.2969577889118119))